In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import *
import matplotlib.pyplot as plt

In [ ]:
def data_split(df, start, end, target_date_col="date"):
    """
    split the dataset into training or testing using date
    :param target_date_col: target date column
    :param end: end date (exclusive)
    :param start: start date (inclusive)
    :param df: (df) pandas dataframe
    :return: (df) pandas dataframe
    """
    data = df[(df[target_date_col] >= start) & (df[target_date_col] < end)]
    data = data.sort_values([target_date_col, "tic"], ignore_index=True)
    data.index = data[target_date_col].factorize()[0]
    return data

def get_pivot(df, value="close"):
    return df.pivot(index="date", columns="tic", values=value)


def normalize(data, min_scale=0, max_scale=1):
    return min_scale + (((data - data.min()) * (max_scale - min_scale)) / (data.max() - data.min()))

def clip(value, min_value=0, max_value=1):
    return max_value if value > max_value else min_value if value < min_value else value

TRAIN_START = "2021-05-11 00:00:00"
TRAIN_END = "2022-05-24 23:59:59"

TEST_START = "2022-05-25 00:00:00"
TEST_END = "2022-11-01 23:59:59"

In [ ]:
data_file = '../../datasets/thesis/crypto_1d_plus.csv'
df = pd.read_csv(data_file, index_col=0)
df2 = data_split(df, TRAIN_START, TEST_END)

In [ ]:
# columns = ['open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_7_sma', 'close_30_sma', 'turbulence']
# columns = ['volume', 'day', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_7_sma', 'close_30_sma', 'turbulence']

In [ ]:
close = get_pivot(df2, 'close')
high = get_pivot(df2, 'high')
macd = get_pivot(df2, 'macd')
bollub = get_pivot(df2, 'boll_ub')
bolllb = get_pivot(df2, 'boll_lb')
volume = get_pivot(df2, 'volume')
rsi = get_pivot(df2, 'rsi_30')
cci = get_pivot(df2, 'cci_30')
dx = get_pivot(df2, 'dx_30')
sma7 = get_pivot(df2, 'close_7_sma')
sma30 = get_pivot(df2, 'close_30_sma')
turbulence = get_pivot(df2, 'turbulence')

### MinMaxScaling
uses `data.min()` and `data.max()` as limits to rescale to `[0, 1]`. 

Clipping: values outside of feature range are clipped to limits or not

In [ ]:
data = close[['BTCUSDT']]
scaler = MinMaxScaler()
scaler.fit(data.values)
print('max', scaler.data_max_)
print('min', scaler.data_min_)
datas = scaler.transform(data.values) # transform to scaled range
invdata = scaler.inverse_transform(datas) # transform back to original
scaler.get_feature_names_out()
scaler.get_params()

In [ ]:
scaler.clip = True
scaler.transform(np.array([10000]).reshape(-1,1))

In [ ]:
scaler.clip = False
scaler.transform(np.array([10000]).reshape(-1,1))

In [ ]:
plt_1 = plt.figure(figsize=(8, 3))
plt.plot(data)
plt.title('original data')
plt.show()

plt_1 = plt.figure(figsize=(8, 3))
plt.plot(datas)
plt.title('min max scaled')
plt.show()

plt_1 = plt.figure(figsize=(8, 3))
plt.plot(invdata)
plt.title('inverse transformed back to original values')
plt.show()

### Defining MaxAbsScaler Limit for Price Values
instead of scaling to max value of data set, we add 1 std to max value of `[high, boll_up]` and use that as max to account for possibly higher values in the future to a certain degree. If markets move outside of this boundary, models have to be rescaled/trained.

used for `open,high,low,close,close_7_sma,close_30_sma`

In [ ]:
max_value_candidates = pd.DataFrame([bollub.max(), high.max()])
max_value_candidates

In [ ]:
price_limits = pd.DataFrame(max_value_candidates.max() + 1 * high.std()).T
price_limits = price_limits.append(price_limits.copy())
price_limits.iloc[1] = 0
price_limits.to_csv('price_normalization_limits.csv')
price_limits

cleaned price limits:

In [ ]:
price_max_norm = pd.read_csv('price_norm_clean.csv', index_col=0)
price_max_norm

### MaxAbsScaler
uses `0` and the max value of the calculated array above as limits to rescale to `[0, 1]`. 


In [ ]:
# price_scaler = MaxAbsScaler()
price_scaler = MinMaxScaler(clip=True)
price_scaler.fit(price_max_norm)

In [ ]:
close_transformed = pd.DataFrame(price_scaler.transform(close), columns=price_max_norm.columns)
sma7_transformed = pd.DataFrame(price_scaler.transform(sma7), columns=price_max_norm.columns)
sma30_transformed = pd.DataFrame(price_scaler.transform(sma30), columns=price_max_norm.columns)
bollub_transformed = pd.DataFrame(price_scaler.transform(bollub), columns=price_max_norm.columns)
bolllb_transformed = pd.DataFrame(price_scaler.transform(bolllb), columns=price_max_norm.columns)

clip test for values outside of bounds:

In [ ]:
pd.DataFrame(price_scaler.transform(pd.DataFrame(np.array([12352] * 5 + [-22] * 5).reshape(1,-1), columns=price_max_norm.columns)), columns=price_max_norm.columns).clip(upper=1, lower=0)

In [ ]:
my_symbol = 'SHIBUSDT'
plt_1 = plt.figure(figsize=(12, 5))
plt.plot(close[my_symbol])
plt.plot(sma30[my_symbol])
plt.plot(sma7[my_symbol])
plt.fill_between(bolllb.index, bolllb[my_symbol], bollub[my_symbol], alpha=0.1)
plt.title('original data')
plt.show()

plt_1 = plt.figure(figsize=(12, 5))
plt.plot(close_transformed[my_symbol])
plt.plot(sma30_transformed[my_symbol])
plt.plot(sma7_transformed[my_symbol])
plt.fill_between(bolllb.index, bolllb_transformed[my_symbol], bollub_transformed[my_symbol], alpha=0.1)
plt.title('transformed')
plt.show()

## Scaling Features [0, 100] to [0, 1]
Features: `RSI, DX` - values outside of bounds get clipped

In [ ]:
normal_minmax = MinMaxScaler(clip=True)
normal_minmax.fit(pd.DataFrame([np.array([100] * 10), np.array([0] * 10)], columns=price_max_norm.columns))
dx_transformed = pd.DataFrame(normal_minmax.transform(dx), columns=price_max_norm.columns).clip(upper=1, lower=0)
rsi_transformed = pd.DataFrame(normal_minmax.transform(rsi), columns=price_max_norm.columns).clip(upper=1, lower=0)

In [ ]:
pd.DataFrame(normal_minmax.transform(pd.DataFrame(np.array([i*20-50 for i in range(10)]).reshape(1,-1), columns=price_max_norm.columns)), columns=price_max_norm.columns).clip(upper=1, lower=0)

In [ ]:
my_symbol = 'SHIBUSDT'
plt_1 = plt.figure(figsize=(8, 3))
plt.plot(dx[my_symbol])
plt.plot(rsi[my_symbol])
plt.title('original data')
plt.show()

plt_1 = plt.figure(figsize=(8, 3))
plt.plot(dx_transformed[my_symbol])
plt.plot(rsi_transformed[my_symbol])
plt.title('transformed')
plt.show()

### Feature Scaling with Quantile Bounds
Clipping `CCI` and `MACD` within [0.05, 0.95] quantiles and rescale to [-1, 1]

In [ ]:
cci_upperbound = np.quantile(cci, .95, axis=0)
cci_lowerbound = np.quantile(cci, .05, axis=0)
cci_limits = pd.DataFrame([cci_lowerbound,cci_upperbound], columns=price_max_norm.columns)
cci_limits.to_csv('cci_limits.csv')

In [ ]:
cci_limits_file = pd.read_csv('cci_limits.csv', index_col=0)
cci_limits_file

In [ ]:
cci_scaler = MinMaxScaler(feature_range=(-1, 1), clip=True)
cci_scaler.fit(cci_limits_file)
cci_transformed = pd.DataFrame(cci_scaler.transform(cci), columns=price_max_norm.columns)

In [ ]:
my_symbol = 'SHIBUSDT'
plt_1 = plt.figure(figsize=(8, 3))
plt.plot(cci[my_symbol])
plt.title('original data')
plt.show()


my_symbol = 'SHIBUSDT'
plt_1 = plt.figure(figsize=(8, 3))
plt.plot(cci_transformed[my_symbol])
plt.title('original data')
plt.show()

In [ ]:
macd_upperbound = np.quantile(macd, .95, axis=0)
macd_lowerbound = np.quantile(macd, .05, axis=0)
macd_limits = pd.DataFrame([macd_lowerbound,macd_upperbound], columns=price_max_norm.columns)
macd_limits.to_csv('macd_limits.csv')

In [ ]:
macd_limits_file = pd.read_csv('macd_limits.csv', index_col=0)
macd_limits_file

In [ ]:
macd_scaler = MinMaxScaler(feature_range=(-1, 1), clip=True)
macd_scaler.fit(macd_limits_file)
macd_transformed = pd.DataFrame(macd_scaler.transform(macd), columns=price_max_norm.columns)

In [ ]:
my_symbol = 'BTCUSDT'
plt_1 = plt.figure(figsize=(8, 3))
plt.plot(macd[my_symbol])
plt.title('original data')
plt.show()

plt_1 = plt.figure(figsize=(8, 3))
plt.plot(macd_transformed[my_symbol])
plt.title('original data')
plt.show()

### Gym Observation Space

In [ ]:
INDICATORS = ["macd", "rsi_30", "cci_30", "dx_30"]
INDICATORS_PLUS = ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_7_sma', 'close_30_sma']

In [ ]:
stock_dimension = 10
state_space = 101
balance_lower = np.array([0])
price_lower = np.array([0] * stock_dimension)
asset_balance_lower = np.array([0] * stock_dimension)
macd_lower = np.array([-1] * 10)
rsi_lower = np.array([0] * 10)
cci_lower = np.array([-1] * 10)
dx_lower = np.array([0] * 10)

boll_ub_lower = np.array([0] * 10)
boll_lb_lower = np.array([0] * 10)
sma7_lower = np.array([0] * 10)
sma30_lower = np.array([0] * 10)


# small indicator list:
lower_bounds = np.concatenate([balance_lower, price_lower, asset_balance_lower, macd_lower, rsi_lower, cci_lower, dx_lower])
lower_bounds.shape

In [ ]:
# large indicator list:
lower_bounds = np.concatenate([balance_lower, price_lower, asset_balance_lower, macd_lower, boll_ub_lower, boll_lb_lower, rsi_lower, cci_lower, dx_lower, sma7_lower, sma30_lower])
lower_bounds.shape

In [ ]:
balance_upper = np.array([np.inf])
price_upper = np.array([1] * stock_dimension)
asset_balance_upper = np.array([np.inf] * stock_dimension)
macd_upper = np.array([1] * 10)
rsi_upper = np.array([1] * 10)
cci_upper = np.array([1] * 10)
dx_upper = np.array([1] * 10)
boll_ub_upper = np.array([1] * 10)
boll_lb_upper = np.array([1] * 10)
sma7_upper = np.array([1] * 10)
sma30_upper = np.array([1] * 10)


# small indicator list:
upper_bounds = np.concatenate([balance_upper, price_upper, asset_balance_upper, macd_upper, rsi_upper, cci_upper, dx_upper])
upper_bounds.shape

In [ ]:
# large indicator list:
upper_bounds = np.concatenate([balance_upper, price_upper, asset_balance_upper, macd_upper, boll_ub_upper, boll_lb_upper, rsi_upper, cci_upper, dx_upper, sma7_upper, sma30_upper])
upper_bounds.shape

In [ ]:
lower_bounds.shape

In [ ]:
from gym import spaces
spaces.Box(low=lower_bounds, high=upper_bounds, shape=(state_space,), dtype=np.float32)

### Normalize Observation Space

In [ ]:
state = get_state(51)
len(state)

In [ ]:
state[1:stock_dimension + 1]

In [ ]:
price_max_norm = pd.read_csv('price_norm_clean.csv', index_col=0)
price_scaler = MinMaxScaler(clip=True)
price_scaler.fit(price_max_norm)

In [ ]:
close_transformed = pd.DataFrame(price_scaler.transform(close), columns=price_max_norm.columns)
sma7_transformed = pd.DataFrame(price_scaler.transform(sma7), columns=price_max_norm.columns)
sma30_transformed = pd.DataFrame(price_scaler.transform(sma30), columns=price_max_norm.columns)
bollub_transformed = pd.DataFrame(price_scaler.transform(bollub), columns=price_max_norm.columns)
bolllb_transformed = pd.DataFrame(price_scaler.transform(bolllb), columns=price_max_norm.columns)

In [ ]:
normal_minmax = MinMaxScaler(clip=True)
normal_minmax.fit(pd.DataFrame([np.array([100] * 10), np.array([0] * 10)], columns=price_max_norm.columns))
dx_transformed = pd.DataFrame(normal_minmax.transform(dx), columns=price_max_norm.columns).clip(upper=1, lower=0)
rsi_transformed = pd.DataFrame(normal_minmax.transform(rsi), columns=price_max_norm.columns).clip(upper=1, lower=0)

In [ ]:
cci_limits_file = pd.read_csv('cci_limits.csv', index_col=0)
cci_scaler = MinMaxScaler(feature_range=(-1, 1), clip=True)
cci_scaler.fit(cci_limits_file)
cci_transformed = pd.DataFrame(cci_scaler.transform(cci), columns=price_max_norm.columns)

In [ ]:
macd_limits_file = pd.read_csv('macd_limits.csv', index_col=0)
macd_scaler = MinMaxScaler(feature_range=(-1, 1), clip=True)
macd_scaler.fit(macd_limits_file)
macd_transformed = pd.DataFrame(macd_scaler.transform(macd), columns=price_max_norm.columns)

In [ ]:
class CryptoEnvNormalizer:

    def __init__(self, stock_dimension, indicators, state_space, df):
        self.stock_dim = stock_dimension
        self.indicators = indicators
        self.state_space = state_space
        self.df = df

        # used for close, sma7, sma30, bollub, bolllb
        self.price_scaler = MinMaxScaler(clip=True)
        self.price_scaler.fit(pd.read_csv('price_norm_clean.csv', index_col=0))
        # used for dx, rsi
        self.normal_minmax = MinMaxScaler(clip=True)
        self.normal_minmax.fit(pd.DataFrame([np.array([100] * 10), np.array([0] * 10)], columns=price_max_norm.columns))
        # used for cci
        self.cci_scaler = MinMaxScaler(feature_range=(-1, 1), clip=True)
        self.cci_scaler.fit(pd.read_csv('cci_limits.csv', index_col=0))
        # used for macd
        self.macd_scaler = MinMaxScaler(feature_range=(-1, 1), clip=True)
        self.macd_scaler.fit(pd.read_csv('macd_limits.csv', index_col=0))

        self.scalers = {
            "close": self.price_scaler,
            "macd": self.macd_scaler,
            "boll_ub": self.price_scaler,
            "boll_lb": self.price_scaler,
            "rsi_30": self.normal_minmax,
            "cci_30": self.cci_scaler,
            "dx_30": self.normal_minmax,
            "close_7_sma": self.price_scaler,
            "close_30_sma": self.price_scaler
        }
        self.transformed_data = self._get_normalizations()

    def get_observation_space(self):
        lower_bounds = self._get_lower_bounds()
        upper_bounds = self._get_upper_bounds()
        return spaces.Box(low=lower_bounds, high=upper_bounds, dtype=np.float64) # shape=(self.state_space,), 

    def _get_lower_bounds(self):
        balance_lower = np.array([0])
        price_lower = np.array([0] * self.stock_dim)
        asset_balance_lower = np.array([0] * self.stock_dim)
        lower_list = [balance_lower, price_lower, asset_balance_lower]

        indicators_zero_bounded = ["boll_ub", "boll_lb", "rsi_30", "dx_30", "close_7_sma", "close_30_sma"]
        indicators_minus_one_bounded = ["macd", "cci_30"]

        for indicator in self.indicators:
            if indicator in indicators_zero_bounded:
                lower_list.append(np.array([0] * self.stock_dim))
            elif indicator in indicators_minus_one_bounded:
                lower_list.append(np.array([-1] * self.stock_dim))
            else:
                raise ValueError(f"Unsupported indicator {indicator}")

        observation_space_lower = np.concatenate(lower_list)

        if len(observation_space_lower) != self.state_space:
            raise ValueError(
                f"LowerObsSpace Bounds expected to be shape {self.state_space} but is {len(observation_space_lower)}")
        return observation_space_lower

    def _get_upper_bounds(self):
        balance_upper = np.array([np.inf])
        price_upper = np.array([1] * self.stock_dim)
        asset_balance_upper = np.array([np.inf] * self.stock_dim)

        upper_list = [balance_upper, price_upper, asset_balance_upper]

        normal_bounded_indicators = ["macd", "boll_ub", "boll_lb", "rsi_30", "cci_30", "dx_30", "close_7_sma",
                                     "close_30_sma"]
        for indicator in self.indicators:
            if indicator in normal_bounded_indicators:
                upper_list.append(np.array([1] * self.stock_dim))
            else:
                raise ValueError(f"Unsupported indicator {indicator}")

        observation_space_upper = np.concatenate(upper_list)

        if len(observation_space_upper) != self.state_space:
            raise ValueError(
                f"UpperObsSpace Bounds expected to be shape {self.state_space} but is {len(observation_space_upper)}")
        return observation_space_upper

    def _get_normalizations(self):
        transformed_data = {}
        # normalize close price + all indicators
        norm_elements = ["close"] + self.indicators

        for item in norm_elements:
            data = self.df.pivot(index="date", columns="tic", values=item)
            transformed = self.scalers[item].transform(data)
            data_normalized = pd.DataFrame(transformed, columns=data.columns)
            transformed_data[item] = data_normalized

        return transformed_data
    
    def get_normalized_state(self, day, original_state):
        balance = original_state[0]
        asset_amounts = original_state[1 + self.stock_dim:1 + (self.stock_dim * 2)]
        close_t = my_norm.transformed_data['close'].loc[day].tolist()
        transformed_indicators = sum((my_norm.transformed_data[tech].loc[day].tolist() for tech in INDICATORS_PLUS), [])
        normalized_state = ([balance] + close_t + asset_amounts + transformed_indicators)
        return normalized_state